In [10]:
import pandas as pd
from flatten_json import flatten
from pandas_geojson import to_geojson, write_geojson, read_geojson

GeoJson for car crashes in New Zealand downloaded from https://catalogue.data.govt.nz/dataset/crash-analysis-system-cas-data5/resource/273b1e7e-4231-443c-ba0b-c92639532d5e

Link to Geojson https://opendata-nzta.opendata.arcgis.com/datasets/NZTA::crash-analysis-system-cas-data-1.geojson?outSR=%7B%22latestWkid%22%3A2193%2C%22wkid%22%3A2193%7D

In [2]:
#Read GeoJson File
geo = read_geojson('Crash_Analysis_System_(CAS)_data.geojson')

In [3]:
#Flatten lists in geojsn
flattened_data = [flatten(feature) for feature in geo['features']]

In [4]:
#Creates a DF
geodf = pd.DataFrame(flattened_data)

In [9]:
#Confirmed Geojsn matches csv data
weatherg = geodf[['properties_fatalCount', 'properties_weatherA']]
concg = weatherg.groupby('properties_weatherA').mean()
concg.sort_values(by='properties_fatalCount')

,properties_fatalCount
properties_weatherA,
Null,0.001648
Hail or Sleet,0.007576
Light rain,0.009645
Snow,0.009715
Fine,0.010649
Heavy rain,0.010859
Mist or Fog,0.018044


In [21]:
#Remove properties_ from column names
for x in geodf.columns:
    geodf = geodf.rename(columns={x: x.replace('properties_','')})
    
for col in geodf.columns:
    print(col)

type
OBJECTID
advisorySpeed
areaUnitID
bicycle
bridge
bus
carStationWagon
cliffBank
crashDirectionDescription
crashFinancialYear
crashLocation1
crashLocation2
crashRoadSideRoad
crashSeverity
crashSHDescription
crashYear
debris
directionRoleDescription
ditch
fatalCount
fence
flatHill
guardRail
holiday
houseOrBuilding
intersection
kerb
light
meshblockId
minorInjuryCount
moped
motorcycle
NumberOfLanes
objectThrownOrDropped
otherObject
otherVehicleType
overBank
parkedVehicle
pedestrian
phoneBoxEtc
postOrPole
region
roadCharacter
roadLane
roadSurface
roadworks
schoolBus
seriousInjuryCount
slipOrFlood
speedLimit
strayAnimal
streetLight
suv
taxi
temporarySpeedLimit
tlaId
tlaName
trafficControl
trafficIsland
trafficSign
train
tree
truck
unknownVehicleType
urban
vanOrUtility
vehicle
waterRiver
weatherA
weatherB
geometry_type
geometry_coordinates_0
geometry_coordinates_1


In [ ]:
#Remove columns with unneeded data
geodf = geodf.drop(columns=['type','geometry_type','crashRoadSideRoad','intersection'])

In [33]:
#rename geojson cordinates
geodf = geodf.rename(columns={'geometry_coordinates_0': 'longitude','geometry_coordinates_1':'latitude'})

In [34]:
geodf.head()

,OBJECTID,advisorySpeed,areaUnitID,bicycle,bridge,bus,carStationWagon,cliffBank,crashDirectionDescription,crashFinancialYear,...,truck,unknownVehicleType,urban,vanOrUtility,vehicle,waterRiver,weatherA,weatherB,longitude,latitude
0,1,NaN,544801.0,0.0,NaN,0.0,2.0,NaN,Null,2011/2012,...,0.0,0.0,Urban,0.0,NaN,NaN,Fine,Null,178.031840,-38.669793
1,2,NaN,528900.0,0.0,NaN,0.0,2.0,NaN,Null,2011/2012,...,0.0,0.0,Urban,0.0,NaN,NaN,Fine,Null,175.264695,-37.785862
2,3,NaN,507000.0,0.0,NaN,0.0,0.0,NaN,Null,2012/2013,...,0.0,0.0,Urban,1.0,NaN,NaN,Fine,Null,174.751715,-36.708328
3,4,NaN,597513.0,0.0,0.0,0.0,2.0,0.0,Null,2011/2012,...,0.0,0.0,Open,0.0,0.0,0.0,Fine,Null,172.394398,-43.609495
4,5,NaN,611500.0,0.0,NaN,0.0,1.0,NaN,Null,2011/2012,...,0.0,0.0,Urban,1.0,NaN,NaN,Fine,Null,168.385299,-46.417826


In [95]:
#creating a new DF so the old one is not modified 
finial = geodf

In [100]:
geodf['crashYear'].unique()

array([2011, 2012, 2013, 2014, 2015, 2016, 2000, 2001, 2002, 2003, 2004,
       2005, 2006, 2007, 2008, 2009, 2010, 2017, 2018, 2019, 2020, 2021,
       2022, 2023])

In [96]:
#Changing values to 0 instead of unkown
fillna = ['bicycle','bridge','bus','carStationWagon','cliffBank','debris','fatalCount','ditch','fence','flatHill','guardRail','houseOrBuilding','kerb','moped','motorcycle','NumberOfLanes','otherVehicleType','parkedVehicle','pedestrian','phoneBoxEtc','postOrPole','schoolBus','seriousInjuryCount','slipOrFlood','strayAnimal','suv','taxi','train','tree','truck','vanOrUtility','vehicle','waterRiver']

In [98]:
#Changing values to 0 instead of unkown
for x in fillna:
    finial[x] = finial[x].fillna(0)


In [99]:
finial

,OBJECTID,advisorySpeed,areaUnitID,bicycle,bridge,bus,carStationWagon,cliffBank,crashDirectionDescription,crashFinancialYear,...,truck,unknownVehicleType,urban,vanOrUtility,vehicle,waterRiver,weatherA,weatherB,longitude,latitude
0,1,NaN,544801.0,0.0,0.0,0.0,2.0,0.0,Null,2011/2012,...,0.0,0.0,Urban,0.0,0.0,0.0,Fine,Null,178.031840,-38.669793
1,2,NaN,528900.0,0.0,0.0,0.0,2.0,0.0,Null,2011/2012,...,0.0,0.0,Urban,0.0,0.0,0.0,Fine,Null,175.264695,-37.785862
2,3,NaN,507000.0,0.0,0.0,0.0,0.0,0.0,Null,2012/2013,...,0.0,0.0,Urban,1.0,0.0,0.0,Fine,Null,174.751715,-36.708328
3,4,NaN,597513.0,0.0,0.0,0.0,2.0,0.0,Null,2011/2012,...,0.0,0.0,Open,0.0,0.0,0.0,Fine,Null,172.394398,-43.609495
4,5,NaN,611500.0,0.0,0.0,0.0,1.0,0.0,Null,2011/2012,...,0.0,0.0,Urban,1.0,0.0,0.0,Fine,Null,168.385299,-46.417826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
821739,1318959,NaN,518600.0,0.0,0.0,0.0,2.0,0.0,South,2022/2023,...,0.0,0.0,Urban,0.0,0.0,0.0,Fine,Null,174.772473,-36.901400
821740,1318960,NaN,554900.0,0.0,0.0,0.0,0.0,0.0,North,2022/2023,...,0.0,0.0,Open,0.0,0.0,0.0,Fine,Null,175.713619,-39.369826
821741,1318961,NaN,517400.0,0.0,0.0,0.0,1.0,0.0,East,2022/2023,...,0.0,0.0,Urban,0.0,0.0,0.0,Fine,Null,174.775535,-36.870158
821742,1318962,NaN,NaN,0.0,0.0,0.0,0.0,0.0,North,2022/2023,...,0.0,0.0,Open,1.0,0.0,0.0,Light rain,Null,174.956021,-37.135856


In [104]:
#filtering out specific years
finial_5 = finial[finial['crashYear'] > 2017]
finial_23 = finial_5[finial_5['crashYear'] > 2022]
finial_5

,OBJECTID,advisorySpeed,areaUnitID,bicycle,bridge,bus,carStationWagon,cliffBank,crashDirectionDescription,crashFinancialYear,...,truck,unknownVehicleType,urban,vanOrUtility,vehicle,waterRiver,weatherA,weatherB,longitude,latitude
80137,141653,NaN,509800.0,0.0,0.0,0.0,3.0,0.0,East,2017/2018,...,0.0,0.0,Urban,0.0,0.0,0.0,Light rain,Null,174.734172,-36.810286
80144,141660,NaN,586606.0,0.0,0.0,0.0,0.0,0.0,Null,2018/2019,...,0.0,0.0,Open,1.0,0.0,0.0,Fine,Null,172.307284,-43.363742
80206,141722,NaN,606100.0,0.0,0.0,0.0,1.0,0.0,South,2017/2018,...,0.0,0.0,Urban,0.0,0.0,0.0,Fine,Null,170.356567,-45.865878
80207,141723,NaN,520402.0,0.0,0.0,0.0,1.0,0.0,North,2018/2019,...,0.0,0.0,Urban,0.0,0.0,0.0,Fine,Null,174.837732,-36.910256
80208,141724,NaN,531720.0,0.0,0.0,0.0,0.0,1.0,East,2017/2018,...,0.0,0.0,Open,1.0,0.0,0.0,Light rain,Null,175.085746,-38.381707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
821739,1318959,NaN,518600.0,0.0,0.0,0.0,2.0,0.0,South,2022/2023,...,0.0,0.0,Urban,0.0,0.0,0.0,Fine,Null,174.772473,-36.901400
821740,1318960,NaN,554900.0,0.0,0.0,0.0,0.0,0.0,North,2022/2023,...,0.0,0.0,Open,0.0,0.0,0.0,Fine,Null,175.713619,-39.369826
821741,1318961,NaN,517400.0,0.0,0.0,0.0,1.0,0.0,East,2022/2023,...,0.0,0.0,Urban,0.0,0.0,0.0,Fine,Null,174.775535,-36.870158
821742,1318962,NaN,NaN,0.0,0.0,0.0,0.0,0.0,North,2022/2023,...,0.0,0.0,Open,1.0,0.0,0.0,Light rain,Null,174.956021,-37.135856


In [106]:
finial_5['crashYear'].unique()

array([2018, 2019, 2020, 2021, 2022, 2023])

In [111]:
#Converted 2023 data to csv
from pathlib import Path  
filepath = Path('finial_23.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
finial_23.to_csv(filepath)  


In [9]:
#conveted 2018-2023 to csv
from pathlib import Path  
filepath = Path('finial_5.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
finial_5.to_csv(filepath)  


In [2]:
#confirming CSVs were red
df_5 = pd.read_csv('finial_5.csv')
df_23 = pd.read_csv('finial_23.csv')
df_5.head()

,Unnamed: 0,OBJECTID,advisorySpeed,areaUnitID,bicycle,bridge,bus,carStationWagon,cliffBank,crashDirectionDescription,...,truck,unknownVehicleType,urban,vanOrUtility,vehicle,waterRiver,weatherA,weatherB,longitude,latitude
0,80137,141653,NaN,509800.0,0.0,0.0,0.0,3.0,0.0,East,...,0.0,0.0,Urban,0.0,0.0,0.0,Light rain,Null,174.734172,-36.810286
1,80144,141660,NaN,586606.0,0.0,0.0,0.0,0.0,0.0,Null,...,0.0,0.0,Open,1.0,0.0,0.0,Fine,Null,172.307284,-43.363742
2,80206,141722,NaN,606100.0,0.0,0.0,0.0,1.0,0.0,South,...,0.0,0.0,Urban,0.0,0.0,0.0,Fine,Null,170.356567,-45.865878
3,80207,141723,NaN,520402.0,0.0,0.0,0.0,1.0,0.0,North,...,0.0,0.0,Urban,0.0,0.0,0.0,Fine,Null,174.837732,-36.910256
4,80208,141724,NaN,531720.0,0.0,0.0,0.0,0.0,1.0,East,...,0.0,0.0,Open,1.0,0.0,0.0,Light rain,Null,175.085746,-38.381707


In [5]:
#removing colums created by the CSV conversion to DF
df_5 = df_5.drop(columns=['Unnamed: 0'])
df_23 = df_23.drop(columns=['Unnamed: 0'])
df_5.head()

,OBJECTID,advisorySpeed,areaUnitID,bicycle,bridge,bus,carStationWagon,cliffBank,crashDirectionDescription,crashFinancialYear,...,truck,unknownVehicleType,urban,vanOrUtility,vehicle,waterRiver,weatherA,weatherB,longitude,latitude
0,141653,NaN,509800.0,0.0,0.0,0.0,3.0,0.0,East,2017/2018,...,0.0,0.0,Urban,0.0,0.0,0.0,Light rain,Null,174.734172,-36.810286
1,141660,NaN,586606.0,0.0,0.0,0.0,0.0,0.0,Null,2018/2019,...,0.0,0.0,Open,1.0,0.0,0.0,Fine,Null,172.307284,-43.363742
2,141722,NaN,606100.0,0.0,0.0,0.0,1.0,0.0,South,2017/2018,...,0.0,0.0,Urban,0.0,0.0,0.0,Fine,Null,170.356567,-45.865878
3,141723,NaN,520402.0,0.0,0.0,0.0,1.0,0.0,North,2018/2019,...,0.0,0.0,Urban,0.0,0.0,0.0,Fine,Null,174.837732,-36.910256
4,141724,NaN,531720.0,0.0,0.0,0.0,0.0,1.0,East,2017/2018,...,0.0,0.0,Open,1.0,0.0,0.0,Light rain,Null,175.085746,-38.381707


In [11]:
df_23['crashYear'].unique()

array([2023])

In [7]:
#removing 2023 from the crash year
df_5 = df_5[df_5['crashYear'] < 2023]

In [8]:
df_5['crashYear'].unique()

array([2018, 2019, 2020, 2021, 2022])

In [15]:
#convert df to GeoJson
json_df_5 = to_geojson(df=df_5, lat='latitude', lon='longitude', properties=df_5.columns)
json_df_23 = to_geojson(df=df_23, lat='latitude', lon='longitude', properties=df_23.columns)
json_df_23

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'OBJECTID': 164691,
    'advisorySpeed': nan,
    'areaUnitID': 538102.0,
    'bicycle': 0.0,
    'bridge': 0.0,
    'bus': 0.0,
    'carStationWagon': 0.0,
    'cliffBank': 0.0,
    'crashDirectionDescription': 'South',
    'crashFinancialYear': '2022/2023',
    'crashLocation1': '15TH AVENUE',
    'crashLocation2': 'CAMERON ROAD',
    'crashSeverity': 'Minor Crash',
    'crashSHDescription': 'No',
    'crashYear': 2023,
    'debris': 0.0,
    'directionRoleDescription': 'North',
    'ditch': 0.0,
    'fatalCount': 0.0,
    'fence': 0.0,
    'flatHill': 'Flat',
    'guardRail': 0.0,
    'holiday': nan,
    'houseOrBuilding': 0.0,
    'kerb': 0.0,
    'light': 'Overcast',
    'meshblockId': 1167700.0,
    'minorInjuryCount': 1.0,
    'moped': 0.0,
    'motorcycle': 0.0,
    'NumberOfLanes': 3.0,
    'objectThrownOrDropped': nan,
    'otherObject': nan,
    'otherVehicleType': 0.0,
    'overBank': nan,
   

In [16]:
#saves GeoJsons to folder
write_geojson(json_df_5, filename='last_5_data.geojson', indent=4)
write_geojson(json_df_23, filename='2023_data.geojson', indent=4)